<a href="https://colab.research.google.com/github/jeochris/DL-Lecture-Study/blob/main/Demo2_tensorflow_2LayerNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Example 2: Two layer neural network
# 2.1 Loading the reuired libraries
import tensorflow as tf
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
!rm -rf /tmp/mylogs 

'rm'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [2]:
# 2.2 Defining the Batch (Sample) size, input dimension, hidden dimension and number of classes
N,D,H,C=64,1000,100,10
# Here, hidden layer=1, hidden nodes=100 => also tuning parameter

In [3]:
# 2.3 Defining the feedforward and loss function
def forward(x):
    a=tf.matmul(x,w1) # w1: input~hidden weight => linear combination
    a_relu=tf.nn.relu(a) # relu function => hidden layer values
    scores=tf.matmul(a_relu,w2) # w2: hidden~output weight
    return scores

def loss_fn(scores, y):
    probs= tf.nn.softmax(scores) # softmax function => output (probability of classes)
    loss= -tf.reduce_sum(y*tf.math.log(probs)) # y: real value ={0,1}
    return loss    


In [5]:
# 2.4 Simulating data matrices = Generate input and output data set
xx=np.random.randn(N,D).astype(np.float32) # dimension D인 N개 input
yy=np.zeros((N,C)).astype(np.float32) # 그거에 대한 output N개 (dimension C) => 10 dimension인 64개 결과
yy[np.arange(N),np.random.randint(C,size=N)]=1 # output 10개 중에 하나를 1로 (classification)
yy
xx
print(xx)
print(yy)

[[-0.56598413  0.23953073  0.01121933 ...  0.18119116  1.3432348
  -0.8247276 ]
 [-0.9243525   0.24899566 -1.0412728  ...  1.6390234  -1.617978
   0.32010147]
 [-0.520543    0.5141282   0.07274141 ... -0.16087748  0.94700056
  -0.26457566]
 ...
 [ 0.23558384 -0.00584414  0.01846702 ... -0.6080901  -1.0966027
  -1.0457603 ]
 [-0.42947787 -0.7071345   2.4097633  ... -0.726892   -0.04989825
  -0.42902738]
 [-0.5078557  -1.2014279  -0.5000859  ... -0.91358835  0.96519935
   0.4983903 ]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 

In [6]:
# 2.5 Defining the Variables; Weight matrices w1 and w2 (initial weight parameter - to be updated)
w1=tf.Variable(1e-3*np.random.randn(D,H).astype(np.float32),name="w1") # input~hidden => D*H개 weight 필요
w2=tf.Variable(1e-3*np.random.randn(H,C).astype(np.float32),name="w2") # hidden~output => H*C개 weight 필요

In [7]:
# 2.7 Perform training (Forward, Back)
learning_rate=1e-2 # tuning parameter
optimizer=tf.keras.optimizers.Adam(learning_rate)

def train_step(x, y):
    with tf.GradientTape() as tape:
        scores = forward(x)
        loss = loss_fn(scores, y) # softmax -> loss function
    gradients = tape.gradient(loss, [w1, w2]) # calc gradients
    optimizer.apply_gradients(zip(gradients, [w1, w2])) # apply gradients to update weight
    return loss, w1, w2
    


**Should try different tuning parameters**
* learning rate
* hidden layer number
* hidden nodes number

=> important when fitting real data sets (project too)

In [8]:
# 2.8 Creating logs for Tensorboard
summary_writer = tf.summary.create_file_writer('/tmp/mylogs')
loss_list = []

for step in range(10): # iterate 10 times
    loss, w1, w2 = train_step(xx, yy)
    loss_list.append(loss.numpy())
    
    with summary_writer.as_default():
        tf.summary.scalar('loss_summary', loss, step=step)
        tf.summary.histogram('w1_hist', w1, step=step)
        tf.summary.histogram('w2_hist', w2, step=step)


In [9]:
# 2.9 Check loss => loss (=error) decreases after iterating
loss_list

[147.36679,
 141.02768,
 95.74078,
 40.37176,
 9.652254,
 1.5853906,
 0.13175254,
 0.0080469735,
 0.00046636222,
 2.7775823e-05]